# Import Libraries

In [6]:
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import requests
import pandas as pd
import csv
import numpy as np
import time
import tensorflow as tf
import scipy
import math
from scipy import stats

# Preprocess Functions

In [7]:
def calc_macd_and_ema(X, display=False):
  '''Calculates the Moving Average Convergence Divergence Indicator
     and the Exponential Moving Average Indicator'''
  macd, ema = [-.1], [1.]

  for i in range(1, len(X)):
    e26 = X[i] * (2 / 27) + X[i-1] * (1 - (2 / 27))
    e12 = X[i] * (2 / 27) + X[i-1] * (1 - (2 / 13))
    m = e12 - e26

    ema.append(e12)
    macd.append(m)

  if display:
    plt.plot(macd)
    plt.title('MACD')
    plt.show()

    plt.plot(ema)
    plt.title('EMA')
    plt.show()

  return np.array(macd), np.array(ema)


def calc_sma(X, window, display=False):
  '''Calculates the Simple Moving Average Indicator'''
  sma = [1.]
  for i in range(1, len(X)):
    if i < window:
      sma.append((sum(X[:i])) / i)
    else: 
      sma.append((sum(X[i-window:i])) / window)

  if display:
    plt.plot(sma)
    plt.title('SMA')
    plt.show()  

  return np.array(sma)

def calc_rsi(X, display=False):
  '''Calculates the Relative Strength Index'''
  look_back = 24
  rsi = np.zeros((len(X)), dtype=np.float32)
  rsi[0] = 50.
  avu = [.001]
  avd = [.001]
  for i in range(1, len(X)):

    if X[i] > X[i-1]:
      avu.append(X[i]/X[i-1])
      if len(avu) > look_back: avu.pop(0)

    else:
      avd.append(X[i]/X[i-1])
      if len(avd) > look_back: avd.pop(0)
    r = (100 - (100 / (1 + (np.mean(avu) / np.mean(avd)))))
      
    if 50. < r < 50.1: rsi[i] = (r) 
    else: rsi[i] = 50.

  if display:
    plt.plot(rsi)
    plt.title('RSI')
    plt.show()

  return np.array(rsi)

def calc_BB(X, ma, display=False):
  '''Calculates the Bollinger Bands Indicator'''
  window = 48
  bbu, bbd = np.zeros((len(X)), dtype=np.float32), np.zeros((len(X)), dtype=np.float32)
  bbu[0] = 0.
  bbd[0] = 0.

  for i in range(1, len(X)):
    if i < window:
      bbu[i] = (ma[i] * np.mean(X[:window]) + 2 * np.std(X[:window]))
      bbd[i] = (ma[i] * np.mean(X[:window]) - 2 * np.std(X[:window]))
    else:
      bbu[i] = (ma[i] * np.mean(X[i-window:i]) + 2 * np.std(X[i-window:i]))
      bbd[i] = (ma[i] * np.mean(X[i-window:i]) - 2 * np.std(X[i-window:i]))

  if display:
    plt.plot(bbu[10000:])
    plt.plot(bbd[10000:])
    plt.title('BBands')
    plt.show()

  return np.array(bbu), np.array(bbd)

In [8]:
def split_sequences(xs, ys, n_steps):
  '''
  Takes an input sequence and splits it into multiple windows
  Input array will be shape [examples, features, 1]
  Output array will be shape [examples, time steps, features, 1]
  '''
  X = np.zeros((xs.shape[0], n_steps, xs.shape[1], 1), dtype=np.float32)
  Y = np.zeros((ys.shape[0], n_steps, 1), dtype=np.float32)
  for i in range(len(xs)):
    end_ix = i + n_steps
    if end_ix > len(xs):
      break

    seq_x = xs[i:end_ix, :]
    seq_y = ys[i:end_ix]
    zeros = np.zeros((n_steps, 7, 1))

    if np.array_equal(seq_x, zeros):
      break
    X[i] = seq_x
    Y[i] = seq_y

  #print(f'CNN X Shape: {np.array(X).shape}')
  #print(f'CNN Y Shape: {np.array(Y).shape}')
  return np.array(X, dtype=np.float32), np.array(Y, dtype=np.float32)

# Get Current Data

In [9]:
def get_current_data(ticker):
  '''
  Takes in a ticker, retrieves price data from alphavantage, and computes MACD, SMA, EMA, and ADX, and returns array
  '''
  tic = time.time()

  #RETRIEVES DATA
  data = {
        "function": "FX_INTRADAY",
        "from_symbol": ticker[:3],
        "to_symbol": ticker[3:],
        "interval": "1min",
        "outputsize" : "compact",
        "apikey": "XXXXXXXXXXXXXXX"
        }

  API_URL = "https://www.alphavantage.co/query"
  response = requests.get(API_URL, params=data)
  keys = response.json()['Time Series FX (1min)'].keys()
  key_list = []
  [key_list.append(i) for i in keys]

  price_list = []

  #Creates list of 100 sequential price values from alphavantage 
  for i in key_list[::-1]:
    value = float(response.json()['Time Series FX (1min)'][i]['1. open'])
    price_list.append(value)
    

  macd, ema = calc_macd_and_ema(price_list, display=False)
  sma = calc_sma(price_list, window=32, display=False)
  rsi = calc_rsi(price_list, display=False)
  bbu, bbd = calc_BB(price_list, sma, display=False)
  bin = np.zeros((macd.shape[0]))
  
  #Reshape data
  price = np.array(price_list)
  macd = macd.reshape(bin.shape[0] , 1)
  sma = sma.reshape(bin.shape[0] , 1)
  ema = ema.reshape(bin.shape[0] , 1)
  rsi = rsi.reshape(bin.shape[0] , 1)
  bbu = bbu.reshape(bin.shape[0] , 1)
  bbd = bbd.reshape(bin.shape[0] , 1)
  price = price.reshape(bin.shape[0] , 1)

  #Normalize the data
  macd = scipy.stats.zscore(macd)
  sma =  scipy.stats.zscore(sma)
  ema = scipy.stats.zscore(ema)
  rsi =  scipy.stats.zscore(rsi)
  bbu = scipy.stats.zscore(bbu)
  bbd = scipy.stats.zscore(bbd)


  xs = np.stack((macd, sma, ema, rsi, bbu, bbd, price), axis=1) #Stack data
  ys = price

  xs, _ = split_sequences(xs, price, 32)
  
  #Keeps empty arrays from being used as inputs
  for i in range(xs.shape[0]-1):
    if xs[i,0,0] == 0.:
      xs = xs[:i-1,...]
      break
    #print(xs[i])

  toc = time.time()

  print(f'Data Collection Time: {toc - tic} s')

  return xs, ys

  


# Live Trade

In [ ]:
def buy(bank, current_price, banked_shares, pred):
  '''Executes a simulated buy order with the given inputs'''
  available_shares = math.floor(bank / current_price)
  bought = math.floor(available_shares * volatility)
  banked_shares += bought
  bank -= (bought * current_price)
  net = bank + banked_shares * current_price
  metrics['bank'].append(bank)
  metrics['net'].append(net)
  metrics['shares'].append(banked_shares)
  metrics['price'].append(current_price)
  metrics['pred'].append(pred)
  return banked_shares, bank, net

def sell(bank, current_price, banked_shares, savings, pred):
  '''Executes a simulated sell order with the given inputs'''
  net = banked_shares * current_price + bank
  liquidity = math.floor(banked_shares * current_price * .9) 
  sold = math.floor(banked_shares * .9)
  if banked_shares >= sold:
    bank += liquidity
    banked_shares -= sold
    net = bank + banked_shares * current_price
    metrics['net'].append(net)

  if bank > (initial * 1.2):
    saved = (bank - (initial * 1.2))
    savings += saved
    bank -= saved
  metrics['bank'].append(bank)
  metrics['shares'].append(banked_shares)
  metrics['price'].append(current_price)
  metrics['pred'].append(pred)
  return banked_shares, bank, savings, net

In [10]:
def live_trade(bank, model, volatility):
  try:
    live_trade = True
    initial = bank
    banked_shares = 50
    savings = 0
    
    while live_trade:
      now = datetime.datetime.now().strftime('%H:%M:%S')
      if (int(now[3:5]) % 1) == 0:
        
        xs, ys = get_current_data('EURUSD')

        tic = time.time()

        pred = model.predict(xs)

        toc = time.time()
        print(f'Elapsed Predict Time: {toc - tic}')

        if np.round(pred[-1]) == 1.:
          banked_shares, bank, net = buy(bank, ys[-1], banked_shares, pred[-1])
          print(f'Bought: {banked_shares} ------ Bank: {bank} ------ Savings: {savings} ------ Current Price: {ys[-1]} ------ Pred: {pred[-1]} ------ Net: {net}')
        elif np.round(pred[-1]) == 0.:
          banked_shares, bank, savings, net = sell(bank, ys[-1], banked_shares, savings, pred[-1])
          print(f'Sold: {banked_shares} ------ Bank: {bank} ------ Savings: {savings} ------ Current Price: {ys[-1]} ------ Pred: {pred[-1]} ------ Net: {net}')
        else:
          print("Passed")
          pass
        time.sleep(60)

      else:
        pass
  #To end simulation terminate the cell and the metrics of the simulation will be plotted.
  except KeyboardInterrupt:
    plt.plot(metrics['bank'])
    plt.title('Bank')
    plt.show()
    plt.plot(metrics['net'])
    plt.title('Net')
    plt.show()
    plt.plot(metrics['shares'])
    plt.title('Banked Shares')
    plt.show()
    plt.figure()
    plt.plot(metrics['price'])
    for i in range(len(metrics['pred'])):
      if np.round(metrics['pred'][i]) == 1.:
        plt.plot(i, metrics['price'][i], '+')
      elif np.round(metrics['pred'][i]) == 0.:
        plt.plot(i, metrics['price'][i], '.')
      else: pass
    plt.show()
    return metrics


In [12]:
metrics = {'bank':[], 
            'net':[], 
            'shares':[],
            'price':[],
            'pred':[]}
bank = 1000
volatility = 0.3
savings = 0

model = new_model = tf.keras.models.load_model(r'C:\Users\Alex Rogers\Downloads\CNN.h5')

metrics = live_trade(bank, model, volatility)

ValueError: Unknown regularizer: L2